In [ ]:
from scapy.all import *

In [ ]:
import pandas as pd
from scapy.all import *
from scapy.layers import http

In [ ]:
import pandas as pd
from scapy.all import *

# Function to extract packet fields and create a DataFrame
def extract_packet_fields(packet, prev_packet_time):
    packet_fields = {
        'frame.len': len(packet),
        'ip.proto': packet.getlayer(IP).proto if IP in packet else None,
        'tcp.srcport': packet.getlayer(TCP).sport if TCP in packet else None,
        'tcp.dstport': packet.getlayer(TCP).dport if TCP in packet else None,
        'udp.srcport': packet.getlayer(UDP).sport if UDP in packet else None,
        'udp.dstport': packet.getlayer(UDP).dport if UDP in packet else None,
        'tcp.seq': packet.getlayer(TCP).seq if TCP in packet else None,
        'frame.time_epoch': packet.time,
        #'tcp.stream': packet.getlayer(TCP).stream if TCP in packet else None,
        #'tcp.stream': packet[TCP].stream if TCP in packet else None,

        'ip.flags.df': packet.getlayer(IP).flags if IP in packet else None,
        
        'frame.encap_type': packet.encap if hasattr(packet, 'encap') else None,
        'frame.cap_len': packet.caplen if hasattr(packet, 'caplen') else None,
        'frame.marked': packet.marked if hasattr(packet, 'marked') else None,
        'eth.lg': packet.len if hasattr(packet, 'len') else None,
        'eth.ig': packet.underlayer.flags & 0x01 if hasattr(packet.underlayer, 'flags') else None,
        'ip.dsfield.dscp': packet[IP].tos if IP in packet else None,
        
        #'ip.flags.rb':1 if packet.getlayer(IP).flags & 4 if IP in packet else 0,
        #'ip.flags.mf':1 if packet.getlayer(IP).flags & 2 if IP in packet else 0,
        'ip.flags.rb': 1 if IP in packet and packet.getlayer(IP).flags & 4 else 0,
        'ip.flags.mf': 1 if IP in packet and packet.getlayer(IP).flags & 2 else 0,

        'frame.time_relative': packet.time - prev_packet_time if prev_packet_time is not None else None,
        'ip.len': packet[IP].len if IP in packet else None,
        'tcp.len': len(packet[TCP].payload) if TCP in packet else None,
        'udp.length': len(packet[UDP].payload) if UDP in packet else None,
        'frame.time_delta': packet.time - prev_packet_time if prev_packet_time is not None else None,
        'ip.hdr_len': packet.getlayer(IP).ihl * 4 if IP in packet else None,
        'tcp.hdr_len': packet.getlayer(TCP).dataofs * 4 if TCP in packet else None,
        'tcp.analysis.bytes_in_flight': packet.getlayer(TCP).window if TCP in packet else None,
        'tcp.time_relative': packet.time - packet.getlayer(TCP).time if TCP in packet else None,
        'tcp.time_delta': packet.time - prev_packet_time if prev_packet_time is not None else None,
        'tcp.analysis.ack_rtt': packet.getlayer(TCP).options[0][1] if TCP in packet and len(packet.getlayer(TCP).options) > 0 else None,
        
       
        'tcp.flags.syn': 1 if packet.getlayer(TCP) and packet[TCP].flags & 0x02 else 0,
        'tcp.flags.ack': 1 if packet.getlayer(TCP) and packet[TCP].flags & 0x10 else 0,
        'tcp.flags.push': 1 if packet.getlayer(TCP) and packet[TCP].flags & 0x08 else 0,
        'tcp.flags.reset': 1 if packet.getlayer(TCP) and packet[TCP].flags & 0x04 else 0,
        'tcp.flags.fin': 1 if packet.getlayer(TCP) and packet[TCP].flags & 0x01 else 0,
        'tcp.flags.urg': 1 if packet.getlayer(TCP) and packet[TCP].flags & 0x20 else 0,
        
        
        'ip.frag_offset': packet[IP].frag if IP in packet else None,
        'ip.ttl': packet[IP].ttl if IP in packet else None,
        'tcp.ack': packet[TCP].ack if TCP in packet else None,
        'tcp.window_size': packet[TCP].window if TCP in packet else None,
        'tcp.flags.cwr': packet[TCP].flags & 0x80 if TCP in packet else None,
        'tcp.window_size_value': packet[TCP].window if TCP in packet else None,
        'tcp.urgent_pointer': packet[TCP].urgptr if TCP in packet else None,
        'tcp.options.nop': packet[TCP].options if TCP in packet else None,
        'tcp.analysis.bytes_in_flight.1': packet[TCP].window if TCP in packet else None,
        'tcp.analysis.push_bytes_sent': 1 if packet.getlayer(TCP) and packet[TCP].flags & TCP_PUSH else 0,
        'icmp.type': packet[ICMP].type if ICMP in packet else None,
        'icmp.code': packet[ICMP].code if ICMP in packet else None,
        'icmp.ident': packet[ICMP].id if ICMP in packet else None,
        'icmp.seq': packet[ICMP].seq if ICMP in packet else None, 
        'http.content_length': packet[http.HTTP].Content_Length if http.HTTP in packet else None,
        
        'data.len': len(packet) if Raw in packet else None,
        'dns.flags.response': packet[DNS].qr if DNS in packet else None,
        'dns.flags.opcode': packet[DNS].opcode if DNS in packet else None,
        'dns.count.queries': packet[DNS].qdcount if DNS in packet else None,
        'dns.count.answers': packet[DNS].ancount if DNS in packet else None,
        'dns.resp.ttl': packet[DNSRR].ttl if DNSRR in packet else None,
        'dns.resp.len': packet[DNSRR].rdlen if DNSRR in packet else None,
        'dns.count.labels': packet[DNS].ancount if DNS in packet else None,
        'tcp.nxtseq': packet[TCP].seq + len(packet[TCP].payload) if TCP in packet else None,
        #'tcp.analysis.flags': packet[TCP].flags if TCP in packet else None,
        'tcp.analysis.flags': 1 if TCP in packet and packet[TCP].flags else 0,

        'udp.stream': packet[UDP].sport if UDP in packet else None,
        'udp.length.bad': len(packet[UDP].payload) if UDP in packet else None,
        'udp.length.bad_zero': 1 if (UDP in packet and len(packet[UDP].payload) == 0) else 0,
        
        'frame.packet_flags_fcs_length': packet[Dot11].FCfield if Dot11 in packet else None,
        'ip.fragment.error': packet[IP].frag if IP in packet else None,
        #'tcp.analysis.flags.1': packet[TCP].flags if TCP in packet else 0,
        'tcp.analysis.flags.1': 1 if TCP in packet and packet[TCP].flags else 0,

        'tcp.analysis.keep_alive': packet[TCP].flags & 0x20 if TCP in packet else None,
        'tcp.analysis.window_full': packet[TCP].flags & 0x02 if TCP in packet else None,
        'tcp.analysis.window_update': packet[TCP].flags & 0x08 if TCP in packet else None,
        'tcp.analysis.zero_window': packet[TCP].flags & 0x00 if TCP in packet else None,
        'tcp.analysis.zero_window_probe': packet[TCP].flags & 0x02 if TCP in packet else None,
        'frame.cap_len.1': packet.caplen if packet.haslayer('Ethernet') else None,

        #'Label' : 'CHAT'
        # Add more fields here as needed
    }

    return packet_fields



In [ ]:

# Read pcap file and extract packet fields
packets = rdpcap('C:\\Users\\Admin\\OneDrive\\Desktop\\MAJOR_PROJECT\\VPN-PCAPS-01\\vpn_aim_chat1a.pcap')
prev_packet_time = None
packet_data = []
for packet in packets:
    packet_data.append(extract_packet_fields(packet, prev_packet_time))
    prev_packet_time = packet.time

# Convert extracted data into DataFrame
df = pd.DataFrame(packet_data)

# Add a 'Label' column with default value None
df['Label'] = None

# Write DataFrame to CSV file
df.to_csv('VOIP_AIMCHAT.csv', index=False)

In [ ]:

import pandas as pd

# Import the CSV file
df = pd.read_csv('VOIP_AIMCHAT.csv')

# Get the number of rows and columns
num_rows = df.shape[0]
num_columns = df.shape[1]

print("Number of rows:", num_rows)
print("Number of columns:", num_columns)


In [ ]:
# List all column names
column_names = df.columns.tolist()
print("Column names:", column_names)